# * 문서 병합

### 1. 문서 전체 높이 측정

In [1]:
import win32com.client

def get_document_total_height(doc):
    """
    문서 전체의 총 세로 길이를 계산합니다 (포인트 단위).
    """
    try:
        # 페이지 수
        num_pages = doc.ComputeStatistics(2)  # 2: wdStatisticPages

        # 각 페이지의 세로 길이
        page_height = doc.PageSetup.PageHeight  # 포인트 단위

        # 문서 전체 높이 = 페이지 수 * 페이지 높이
        total_height = num_pages * page_height

        return total_height

    finally:
        print("문서 전체 높이 확인 완료")

### 2. 사용된 폰트와 크기

In [4]:
def get_document_fonts(doc):
    """
    문서에서 사용된 폰트와 크기 정보를 추출합니다.
    """
    try:
        fonts = {}
        for paragraph in doc.Paragraphs:
            font_name = paragraph.Range.Font.Name
            font_size = paragraph.Range.Font.Size

            # 폰트 이름과 크기를 저장
            if font_name not in fonts:
                fonts[font_name] = set()
            fonts[font_name].add(font_size)

        return {font: list(sizes) for font, sizes in fonts.items()}

    finally:
        print("폰트/크기 확인 완료")

### 3. 자간 및 행간 정보

In [5]:
def get_document_spacing_info(doc):
    """
    문서 내 글자의 자간(Spacing) 및 행간(LineSpacing) 정보를 추출합니다.
    """

    try:
        spacing_info = []
        for paragraph in doc.Paragraphs:
            line_spacing = paragraph.LineSpacing  # 행간
            for character in paragraph.Range.Characters:
                char_spacing = character.Font.Spacing  # 자간
                spacing_info.append({
                    "line_spacing": line_spacing,
                    "char_spacing": char_spacing
                })

        return spacing_info

    finally:
        print("자간 확인 완료")

### 4. 위/아래 여백

In [6]:
import win32com.client

def get_document_margins(doc):
    """
    문서의 위, 아래 여백 정보를 추출합니다 (포인트 단위).
    """
    try:
        # 위쪽 및 아래쪽 여백 가져오기
        top_margin = doc.PageSetup.TopMargin    # 위 여백 (pt 단위)
        bottom_margin = doc.PageSetup.BottomMargin  # 아래 여백 (pt 단위)

        return {
            "top_margin": top_margin,
            "bottom_margin": bottom_margin
        }

    finally:
        print("위/아래 여백 확인")

In [8]:
import win32com.client
import os

def calculate_content_height(doc):
    """
    Word 문서 내용의 전체 높이를 계산합니다.
    (텍스트 높이 + 행간 + 이미지 높이)
    """

    try:
        # 전체 높이를 계산하기 위한 변수
        total_text_height = 0
        total_image_height = 0

        # 텍스트 높이와 행간 계산
        for paragraph in doc.Paragraphs:
            font_size = paragraph.Range.Font.Size  # 폰트 크기 (pt)
            line_spacing = paragraph.LineSpacing  # 행간 (pt)
            
            # 문단 내 줄 수 계산
            line_count = paragraph.Range.ComputeStatistics(1)  # 1은 wdStatisticLines
            
            # 문단 높이 계산
            paragraph_height = (font_size * line_count) + (line_spacing * (line_count - 1))
            total_text_height += paragraph_height

        # 이미지 높이 계산
        for shape in doc.InlineShapes:
            total_image_height += shape.Height

        # 총 높이 계산
        total_height = total_text_height + total_image_height

        return {
            "text_height": total_text_height,
            "image_height": total_image_height,
            "total_height": total_height
        }
        
    except:
        print("문서 내용 전체 확인 오류류")
    
    finally:
        print("문서 전체 내용 높이 확인")

In [9]:
file_list = ["Q_1", "Q_2", "Q_3", "Q_4"]

# .ipynb 경로
base_path = os.getcwd()

word = win32com.client.Dispatch("Word.Application")
word.Visible = False

for file in file_list:
    doc_path = os.path.join(base_path, f"word_files\{file}.docx")
    # doc_path = r"D:\YearDreamSchool-D\python_projects\msword_pjt\word_files\Q_1.docx"
    
    # 문서 파일 열기
    doc = word.Documents.Open(doc_path)


    #--------- 문서 전체 높이 확인 ---------
    total_height = get_document_total_height(doc)

    print(f"문서 전체 높이: {total_height} pt")

    #--------- 문서 전체 높이 확인 ---------
    fonts = get_document_fonts(doc)

    print("문서에서 사용된 폰트와 크기:")
    for font, sizes in fonts.items():
        print(f" - {font}: {sizes}")

    #--------- 문서 전체 높이 확인 ---------
    spacing_info = get_document_spacing_info(doc)

    print("문서 자간 및 행간 정보:")
    for info in spacing_info[:5]:  # 첫 5개만 출력
        print(f" - 행간: {info['line_spacing']} pt, 자간: {info['char_spacing']} pt")

    #--------- 문서 전체 높이 확인 ---------
    margins = get_document_margins(doc)

    print("문서 여백 정보:")
    print(f" - 위 여백: {margins['top_margin']} pt")
    print(f" - 아래 여백: {margins['bottom_margin']} pt")

    #--------- 문서 전체 높이 확인 ---------
    content_height = calculate_content_height(doc)

    print("문서 내용 높이 정보:")
    print(f" - 텍스트 높이: {content_height['text_height']} pt")
    print(f" - 이미지 높이: {content_height['image_height']} pt")
    print(f" - 총 높이: {content_height['total_height']} pt")

    print("\n", "--------------------------------------------------------", "\n")
    
    # 문서 닫기
    doc.Close(False)

word.Quit()

문서 전체 높이 확인 완료
문서 전체 높이: 792.0 pt
폰트/크기 확인 완료
문서에서 사용된 폰트와 크기:
 - Arial: [11.0]
자간 확인 완료
문서 자간 및 행간 정보:
 - 행간: 13.800000190734863 pt, 자간: 0.0 pt
 - 행간: 13.800000190734863 pt, 자간: 0.0 pt
 - 행간: 13.800000190734863 pt, 자간: 0.0 pt
 - 행간: 13.800000190734863 pt, 자간: 0.0 pt
 - 행간: 13.800000190734863 pt, 자간: 0.0 pt
위/아래 여백 확인
문서 여백 정보:
 - 위 여백: 56.70000076293945 pt
 - 아래 여백: 56.70000076293945 pt
문서 전체 내용 높이 확인
문서 내용 높이 정보:
 - 텍스트 높이: 148.60000038146973 pt
 - 이미지 높이: 187.9499969482422 pt
 - 총 높이: 336.5499973297119 pt

 -------------------------------------------------------- 

문서 전체 높이 확인 완료
문서 전체 높이: 792.0 pt
폰트/크기 확인 완료
문서에서 사용된 폰트와 크기:
 - Arial: [11.0]
 - : [11.0]
자간 확인 완료
문서 자간 및 행간 정보:
 - 행간: 13.800000190734863 pt, 자간: 0.0 pt
 - 행간: 13.800000190734863 pt, 자간: 0.0 pt
 - 행간: 13.800000190734863 pt, 자간: 0.0 pt
 - 행간: 13.800000190734863 pt, 자간: 0.0 pt
 - 행간: 13.800000190734863 pt, 자간: 0.0 pt
위/아래 여백 확인
문서 여백 정보:
 - 위 여백: 56.70000076293945 pt
 - 아래 여백: 56.70000076293945 pt
문서 전체 내용 높이 확인
문서 내용 높